In [ ]:
import pandas as pd
import json
df = pd.read_csv('fixing_institutions/data.csv')
df.columns

with open('../church_dictionary/church_names_dict.json', 'r') as f:
    church_dict = json.load(f)

church_dict

In [ ]:
from functools import reduce
import re
# extracting all church that have been standardized by Isa (meaning with a QXXX code in the value):
reg = r'Q[0-9]+'
cols_of_interest = ['owner_standardised','curr_entity_standardized', 'curr_entity',
       'old_entity', 'old_entity_standardized']


# function that extract the group from the regex
def extract_group_from_regex(t, reg):
    m = re.search(reg, t)
    if m:
        return m.group(1)
    else:
        return None
# test the function

def extract_wikidata_id_and_rest_from_mention(t):
    m = extract_group_from_regex(t, r'Q([0-9]+)')
    t = t.replace('Q' + m, '')
    t = t.replace('(','').replace(')','')
    return f'Q{m}', t.strip()

def is_wikidata_mentioned(t):
    m = re.search(r'Q[0-9]+', t)
    if pd.isna(t):
        return False
    if m:
        return True
    else:
        return False

val_of_interest = df[df[cols_of_interest].apply(lambda x: x.str.contains(reg, na=False))][cols_of_interest].values
wiki_vals = reduce(lambda x, ys: x.union(set([y for y in ys if type(y) is str and re.search(reg, y) is not None])), val_of_interest, set())
dfwv = pd.DataFrame(wiki_vals)
dfwv.columns = ['val']
dfwv['wiki_val'] = dfwv['val'].apply(lambda x: extract_group_from_regex(x, r'Q([0-9]+)'))
dfg = dfwv.groupby('wiki_val').agg(list).reset_index()
dfg['church_short_name'] = ''
# dfg.to_csv('church_sn.csv', index=False)

In [ ]:
# scuola 

# 1270723,"['Scuola Grande di San Rocco (Q1270723)', 'Scuola Grande di San Rocco, (Q1270723)']",Scuola Grande di San Rocco
# 1273362,['Scuola Grande di San Marco (Q1273362)'],Scuola Grande di San Marc
# 2261753,['Scuola dei Mercanti (Q2261753)'],Scuola dei Mercanti
# 1286515,['Scuola Grande di Santa Maria della Misericordia (Q1286515)'],Scuola Grande di Santa Maria della Misericordia
# 130608287,['Scuola Grande di San Giovanni Evangelista (Q130608287)'],Scuola Grande di San Giovanni Evangelista

# rights of use data creation

In [ ]:
def right_of_use_from_row_info(r: pd.Series) -> str:
    own_std = r.owner
    if pd.isna(own_std):
        return own_std
    if 'benefi' in own_std.lower():
        return 'BENEFICIO'
    if 'prebenda' in own_std.lower():
        return 'PREBENDA'
    if r.is_people:
        return 'PROPRIO'
    if 'commissar' in own_std.lower():
        return 'COMISSARIA'
        # put in owner supplementary whenever there is "COMMISSARIA" in the owner (dead persone stating in their )
    else:
        return 'PUBBLICO'
    
df['right_of_use'] = df.apply(right_of_use_from_row_info, axis=1)

# patching bunch of acquisition data (old entity)

In [ ]:
## Manual adjustements 
def auto_update(df, unique_id, old_entity, curr_entity, owner_supplementary = None):
    if isinstance(unique_id, int):
        unique_id = [unique_id]
    for uid in unique_id:
        if old_entity:
            df.loc[df['unique_id'] == uid, 'old_entity'] = old_entity
        if curr_entity:
            df.loc[df['unique_id'] == uid, 'curr_entity'] = curr_entity
        if owner_supplementary:
            df.loc[df['unique_id'] == uid, 'owner_supplementary'] = owner_supplementary
    return df

manual_fix_vals_2_vals = [
# BENEFICIO della Chiesa Parocchiale di S. Paterniano spettante al Paroco ora concentrata in S. Lucca attualmente goduto dal Paroco Sacerdote Gaetano SANDRINELLI,,,[1774]
(1774, 'Paterniano', 'Lucca', 'Gaetano SANDRINELLI'),
# BENEFICIO intitolato del primo Prete della Chiesa sopressa di S. Paterniano concentrata in S. Lucca attualmente goduto dal Sacerdote Antonio Osti,,,[1773]
(1773, 'Paterniano', 'Lucca', 'Antonio Osti'),
# Beneficio del Primo Prete di S. Maria Nova ora in S. Giovanni Grisostomo attualmente goduto dal Sacerdote Molin Beltrame,Maria Nova,,[4733]
(4733, 'Maria Nova', 'Giovanni Grisostomo', 'Molin Beltrame'),
# "CAPITOLO dei Sacerdoti della Chiesa Parrocchiale di S. Eustachio volgarmente detto di S. Stae, altre volte di S. Giovanni Decolato vulgò S. Zan Degolà Parochia sopressa",,,[17113]
(17113, 'Stae', 'Giovanni Decolato'),
# CAPITOLO dei Sacerdoti della Chiesa di S. Eustachio altre volte di S. Giovanni Decolato,,"[17119, 17118]"
([17119, 17118], 'Giovanni Decolato', 'Stae'),
# "CAPITOLO dei Sacerdoti della Chiesa di S. Eustachio, volgamente detta di S. Stae altre volte della sopressa Parochia di S. Giovanni Decolato, volgarmente detta di S. Zan Degolà",,,[17328]
(17328, 'Giovanni Decolato', 'Stae'),
# "CAPITOLO dei Sacerdoti della soppressa Parrocchia dei Santi Ubaldo ed Agata, volgarmente detta di S. Boldo unita alla Chiesa Parrocchiale di S. Giacomo Apostolo, volgarmente detta S. Giacomo dall'Orio",,,"[16606, 16605]"
([16606, 16605], 'Boldo', "Giacomo dall'Orio"),
# "CAPITOLO dei Sacerdoti di S. Agostino, unito alla Parrocchiale di S. Paolo, volgarmente detta di S. Polo",,,[16546]
(16546, 'Agostin', 'Polo'),
# "CAPITOLO dei Sacerdoti di S. Steffano Prete, volgarmente detto S. Stin, ora unito alla Chiesa di S. Tommaso Apostolo volgarmente detta di S. Tomà",,,[16463]
(16463, 'Stin', 'Tomà'),
# PREBENDA Parrocchiale di S. Eustachio altre volte S. Giovanni Decolato,,[17122, 17123, 17121]
([17122, 17443, 17123, 17121], 'Giovanni Decolato', 'Stae'),
# CAPITOLO della Chiesa Patriarcale e Parrocchiale di S. Pietro ora in S. Marco,,,"[13506, 13505, 13508, 13507, 13509]"
([13506, 13505, 13508, 13507, 13509], 'Pietro di Castello', 'Marco'),
# "CHIESA Sussidiaria di S. Giovanni Novo agregato alla Chiesa di S. Marco,,,[3929]"
(3929, 'Giovanni Nuovo', 'Marco'),
# "EX - PREBENDA  EX - PREBENDA Parochiale di S. Gregorio, ora Prebenda di S. Agnese",,,"[22410, 22412]"
# "EX - PREBENDA Parochiale di S. Gregorio, ora Prebenda di S. Agnese",,,[22409]
([22409, 22410, 22412], 'Gregorio', 'Agnese'),
# PREBENDA Parrocchiale di S. Eustachio altre volte di S. Maria Mater Domini,,,"[16687, 16689]" 
([16687, 16689], ' Maria Mater Domini', 'Stae'),
#"PREBENDA Parrocchiale di S. Paolo, altre volte di S. Agostino",,,"[16555, 16556, 16557, 16554]"
# "PREBENDA Parrocchiale di S. Paolo, volgarmente detta S. Polo, altre volte di S. Agostin",,,[16545]
([16545, 16555, 16556, 16557, 16554], 'Agostino', 'Paolo'),
# Prebenda Parrocchiale della Chiesa di San Paolo Appostolo altre volte di San Agostino,,,[23463]
([23463], 'Agostino', 'Polo'),
# "Prebenda Parrochiale di San Silvestro, altre volte di San Giovanni Elemosinario",,,[23435]
(23435, ' Giovanni Elemosinario', 'Silvestro'),
# "RR. MM. Cappucine  degli Ognissanti, altre volte di S. Antonio di Castello",,,[21536]
# "RR. MM. Cappucine degli Ognissanti, altre volte di S. Antonio di Castello",,,"[21532, 21533, 21534]"
([21536, 21532, 21533, 21534], 'Antonio di Castello', 'Ognissanti'),
([19654, 19662, 19663, 19655, 19656, 19657, 19658, 19659, 19660], 'Raffaele Arcangelo', 'Gesu\' e Maria'),
# Spazio di terra altre volte Cimiterio di Santa Maria della Ca' Grande dei Frari,,,[23499]
(23499, '', 'Maria della Ca\' Grande'),
# "RR. MM. Terziarie Francescane dette Pizzochere in S. Raffaele Arcangelo, ora del Gesu' e Maria",Raffaele Arcangelo,,[20071]
(20071, 'Raffaele Arcangelo', 'Gesu\' e Maria'),
# "RR. MM. controscritte Reverende Madri Terziarie Francescane dette Pizzochere in S. Raffaele Arcangelo, ora concentrate nel Monastero del Gesu', e Maria",,,[19661]
(19661, 'Raffaele Arcangelo', 'Gesu\' e Maria'),
([391, 392, 393], 'Stefano', 'REGIO DEMANIO', 'Borò del Genio'),
# Oratorio altre volte Scola di Santa Maria del Carmine,,,[23546]
(23546, '', 'Santa Maria del Carmine'),
# Luogo annesso alla  Chiesa altre volte Scola dei Mercanti Campanile della Chiesa di San Silvestro,,,[23443]
([23443],'Chiesa di San Silvestro', '', 'Scola dei Mercanti')
]


In [ ]:
from Levenshtein import ratio
def church_matching(to_match) -> (str, str):
    """
    uses light NLP to match the church name with the dictionary. Returns the wikidata id and the standardised church name
    """
    best_matches = []
    for k, v in church_dict.items():
        for c in v['church_names']:
            r = ratio(to_match.lower(), c.lower())
            if r > 0.8:
                best_matches.append((r, k, v['standard_name']))
    best_matches = sorted(best_matches, key=lambda x: x[0], reverse=True)
    if len(best_matches) > 0:
        return best_matches[0][1], best_matches[0][2]
    else:
        return None, None

quick_church_fmt = lambda c,v: f"{v} ({c})" if c else None    
res = []
for vals in manual_fix_vals_2_vals:
    curr_entity = church_matching(vals[1])
    curr_entity = quick_church_fmt(curr_entity[0], curr_entity[1]) if curr_entity else vals[1]
    old_entity = church_matching(vals[2])
    old_entity = quick_church_fmt(old_entity[0], old_entity[1]) if old_entity else vals[2]
    if len(vals) > 3:
        res.append((vals[0], curr_entity, old_entity, vals[3]))
    else:
        res.append((vals[0], curr_entity, old_entity))

for vals in res:
    df = auto_update(df, *vals)

# manual standardisation Cédric 

In [ ]:
dfc = pd.read_csv('fixing_institutions/standardizing_institution_fixing.csv')
dfc = dfc[dfc['church_name'].notna() | dfc['owner_supplementary'].notna()]
dfc['church_standardized'] = dfc['church_name'].apply(lambda x: quick_church_fmt(*church_matching(x)) if not pd.isna(x) else x)
# dfc[dfc['church_standardized'].isna()].groupby('church_name').agg(list).reset_index().to_csv('t.csv')
dfc

for i, r in dfc.iterrows():
    df = auto_update(df, r['unique_id'], None, r['church_standardized'], r['owner_supplementary'])
    

# manual standardisation Isabella

In [ ]:
dfi = pd.read_csv('fixing_institutions/last_institutions_unnormalized.csv')
for i, r in dfi.iterrows():
    df = auto_update(df, r['unique_id'], r['Old_entity'], r['normalized_version'], r['Owner supplementary'])

# merging curr_entity with "owner_standardised".
and other redundant columns

In [ ]:
church_matches = {
    "fabrica della chiesa di s. geremia": 'geremia',
    'Abbazia della misericordia (Q1059409)': 'Abbazia della misericordia',
    'fabrica della chiesa di san vidal': 'vidal',
    'capitolo di san geremia': 'geremia',
    'piovano di san giacomo di rialto': 'giacomo di rialto',
    'monastero di s. lucia': 'lucia',
    'chiesa di san zuane': 'giuliano',
    'scuola di san rocco': 'scuola di san rocco',
    'scola di san rocco': 'scola di san rocco',
    'chiesa di s. maria zobenigo': 'maria zobenigo',
    'scola di santa maria della carita': 'scola di santa maria della carita',
    'capitolo di s. benedetto': 'benedetto',
    'pieve di san giovanni novo': 'giovanni nuovo',
    'pieve di san martino': 'martino',
    'pieve di san silvestro': 'silvestro',
    'fabrica della chiesa di s. maria zobenigo': 'maria zobenigo',
    'chiesa di s. angelo': 'angelo',
    'fabrica della chiesa di san samuel': 'samuele',
    'capitolo di san moise': 'moise', 
    'capitolo della chiesa di san nicolo': 'nicolo',
    'CHIESA di S. Steffano officiata dai Padri Agostiniani concentrati alli Carmini': 'stefano',
    'CHIESA di S. Vitale (Locali Sagri annessi)': 'vitale',
    'Monastero di San Daniele': 'daniele',
    'Monastero di San Rocco': 'rocco',
    'Monastero di Santa Marta': 'marta',
    'capitolo della chiesa di san nicolo': 'nicolo',
    'capitolo della chiesa di san vidal': 'vidal',
    'capitolo di s. benedetto': 'benedetto',
    'capitolo di san geremia': 'geremia',
    'capitolo di san moise': 'moisé',
    'capitolo et fabrica di santa maria maddalena': 'maddalena',
    'chiesa di s. angelo': 'angelo',
    'chiesa di s. maria zobenigo': 'maria zobenigo',
    'chiesa di san zuane': 'giuliano',
    'chiesa di santa agnese': 'agnese',
    'fabrica della chiesa di s. geremia': 'geremia',
    'fabrica della chiesa di s. maria zobenigo': 'maria zobenigo',
    'fabrica della chiesa di san samuel': 'samuele',
    'fabrica della chiesa di san vidal': 'vidal',
    "monastero degl'angeli di murano": 'Angeli di Murano',
    'monastero degli angeli di muran': 'Angeli di Murano',
    'monastero del corpus domini': 'corpus domini',
    'monastero della celestia': 'celestia',
    'monastero delle reverende monache de ogni santi': 'ognissanti',
    'monastero delle vergini': 'vergini',
    'monastero di Santa Giustina, (Q3585413)': 'giustina',
    'monastero di s. alvise': 'alvise',
    'venerando monastero di santa maria della celestia': 'celestia',
    'reverende monache del corpus domini': 'corpus domini',
    'monastero di san lorenzo': 'lorenzo',
    'monastero di santa chiara': 'chiara',
    'monastero di santa chiara di venezia': 'chiara',
    'monastero di s. anna': 'anna',
    'monastero di s. antonio di torcello': 'antonio',
    'monastero di s. lucia': 'lucia',
    'monastero di san domenico': 'domenico',
    'monastero di san domenico di castello': 'domenico',
    'monastero di san gerolamo': 'girolamo',
    'monastero di san girolamo': 'gerolamo',
    'monastero di san giovanni di torcello': 'giovanni di torcello',
    'monastero di san zaccaria': 'zaccaria',
    "monastero di sant'andrea": 'andrea della zirada',
    'venerando monastero di san caterina': 'caterina',
    "venerando monastero di sant'alvise": 'alvise',
    'reverendo capitolo di santa maria formosa': 'maria formosa',
}


# 'Confraternita di San Nicolò di Bari': 'nicolo di castello',

owner_entity_std_churches = {k: quick_church_fmt(*church_matching(v)) for k, v in church_matches.items()}
owner_entity_std_churches

In [ ]:
for i, r in df.iterrows():
    # merging curr_entity.
    if not pd.isna(r['curr_entity']):
        to_replace = r['curr_entity'] if pd.isna(r['curr_entity_standardized']) else r['curr_entity_standardized']
        if to_replace in owner_entity_std_churches:
            to_replace = owner_entity_std_churches[to_replace]
        df.loc[i, 'owner_standardised'] = to_replace
    # fetching the right class
    if not pd.isna('owner_standardized_entity_standardized_class'):
        df.loc[i, 'owner_standardised_class'] = r['owner_standardized_entity_standardized_class']
    if not pd.isna(r['owner_entity_standardized_class']):
        df.loc[i, 'owner_standardised_class'] = r['owner_entity_standardized_class']
    # fetching back the owner identity when it was better standardized
    if not pd.isna(r['owner_entity_standardized']):
        to_replace = r['owner_entity_standardized']
        if not is_wikidata_mentioned(r['owner_standardised']):
            if to_replace in owner_entity_std_churches:
                to_replace = owner_entity_std_churches[to_replace]
        df.loc[i, 'owner_standardised'] = to_replace
    # then applying the church standardization to the old_entity
    old_entity_std = r['old_entity_standardized'] if not pd.isna(r['old_entity_standardized']) else r['old_entity']
    if old_entity_std in owner_entity_std_churches:
        old_entity_std = owner_entity_std_churches[r['old_entity_standardized']]
    df.loc[i, 'old_entity'] = old_entity_std
df = df.drop(columns=['curr_entity_standardized', 'curr_entity', 'owner_standardized_entity_standardized_class', 'owner_standardized_entity_standardized', 'owner_entity_standardized', 'owner_entity_standardized_class'])

# taking care of the Comissaria thing

In [ ]:
# df[df['right_of_use'] == 'COMISSARIA'][['unique_id', 'owner', 'owner_standardised', 'old_entity']].to_csv('commissaria.csv', index=False)
# only 20 entries, low enought I can do them manually
commissaria_vals = {
919: "Bonaventura",
967: "SPIRONI Andrea",
7180: "Mariana",
7181: "Mariana",
9436: "ZANONI Giovanni",
9443: "ZANONI Giovanni",
11205: "PREFETTURA dell'Adriatico",
11420: "PREFETTURA dell'Adriatico",
14327: "Procuratia di S. Marco",
14328: "Procuratia di S. Marco",
14330: "Procuratia di S. Marco",
14331: "Procuratia di S. Marco",
19009: "BARBARO",
20369: "BALBI VALIER Marco q.m. Girolamo Maria",
20874: "DAPONTE",
23272: "COSTA Giacoma",
23273: "COSTA Giacoma",
}
for i, v in commissaria_vals.items():
    df = auto_update(df, i, None, None, v)

# Splitting Wikidata standardisation in its own colums for owner & old entity

In [ ]:
df['owner_wd'] = None
df['old_entity_wd'] = None
for i, r in df.iterrows():
    own = r['owner_standardised']
    if not pd.isna(own) and is_wikidata_mentioned(own):
        wd, entity = extract_wikidata_id_and_rest_from_mention(own)
        if wd in church_dict:
            entity = church_dict[wd]['standard_name']
        df.loc[i, 'owner_wd'] = wd
        df.loc[i, 'owner_standardised'] = entity
    old = r['old_entity']
    old_wd = None
    old_entity = None
    if not pd.isna(old) and is_wikidata_mentioned(old):
        old_wd, old_entity = extract_wikidata_id_and_rest_from_mention(old)
        df.loc[i, 'old_entity'] = old_entity
    else:
        old_std = r['old_entity_standardized']
        if not pd.isna(old_std) and is_wikidata_mentioned(old_std):
            old_wd, old_entity = extract_wikidata_id_and_rest_from_mention(old_std)
    if old_wd and old_entity:
        if old_wd in church_dict:
            old_entity = church_dict[old_wd]['standard_name']
        df.loc[i, 'old_entity_standardized'] = old_entity
        df.loc[i, 'old_entity_wd'] = old_wd

df['owner_standardised'] = df['owner_standardised'].apply(lambda x: x.lower() if not pd.isna(x) else None)
df['old_entity_standardized'] = df['old_entity_standardized'].apply(lambda x: x.lower() if not pd.isna(x) else None)

# misc. edits

In [ ]:
# manual correction (owner to quality match):

# id 23339: owner in quality, "S. Maurizio" in owner. 
# id 23401 23620
# id: 11778

def move_owner_to_quality(df, uid, new_owner, own_std = None, own_wd = None):
    """
    Move the owner to the quality column and set the owner to the new_owner
    """
    if isinstance(uid, int):
        uid = [uid]
    for u in uid:
        if own_std:
            df.loc[df['unique_id'] == u, 'owner_standardised'] = own_std
        if own_wd:
            df.loc[df['unique_id'] == u, 'owner_wd'] = own_wd
        df.loc[df['unique_id'] == u, 'quality'] = df.loc[df['unique_id'] == u, 'owner']
        df.loc[df['unique_id'] == u, 'owner'] = new_owner
    return df


df = move_owner_to_quality(df, 23339, 'Parrocchia di S. Maurizio', 'Chiesa di san maurizio', 'Q251686')
df = move_owner_to_quality(df, [23401, 23620], 'Parrocchia di S. Benedetto', 'Chiesa di san beneto', 'Q3585318')
# a regarder avec Isabella. 
# df = move_owner_to_quality(df, 11778, 'Parrocchia di S. Giovanni Battista', 'Chiesa di san giovanni battista', 'Q3585318')

#3,Giovanni Zalirani,"['CLERO dei Preti di Venezia come sopra', 'CLERO della Congregazione dei Preti di Venezia rappresentati dal Sacerdote Giovanni Zalirani Procuratore', 'CLERO della Congregazione dei Preti di Venezia rappresentati dal Sacerdote Giovanni Zalirani Procuratore di detto Clero']","[nan, nan, nan]","['[7371]', '[7368]', '[7367]']","[None, None, None]"
# a standardiser: Congregazion dei Preti di Venezia (matcher avec l'entrée) et owner_supplemetary: Giovanni Zalirani

# entité ad-hoc:
# 5,Scuola del Santissimo di S. Moisè,['Benefizio della Scuola del Santissimo di S. Moisè'],[nan],['[7801]'],[None]


In [ ]:
religious_entities_trig = ['chiesa', 'monestaro']
cols_to_check = {'owner_standardised': 'owner_standardised_class', 'old_entity_standardized': 'old_entity_standardized_class'}

print('owner_standardised_class', df['owner_standardised_class'].isna().sum())
print('old_entity_standardized_class', df['old_entity_standardized_class'].isna().sum())

for i, row in df.iterrows():
    for religious_entity in religious_entities_trig:
        for col, class_col in cols_to_check.items():
            if not pd.isna(row[col]) and religious_entity in row[col].lower() and pd.isna(row[class_col]):
                df.loc[i, class_col] = 'religious_entities'

print('owner_standardised_class', df['owner_standardised_class'].isna().sum())
print('old_entity_standardized_class', df['old_entity_standardized_class'].isna().sum())

In [ ]:
appreciated_order = list([
    'unique_id', 'owner', 'owner_standardised', 
    'owner_standardised_class', 'owner_wd', 'old_entity',
      'old_entity_standardized', 'old_entity_standardized_class', 'old_entity_wd',
     'right_of_use', 'owner_supplementary',
     'geometry_id', 'parcel_number', 'sub_parcel_number', 'place', 'house_number',
       'quality',  'district_acronym',
       'austrian_cadaster_correspondance',
       'austro_italian_cadaster_correspondance','area', 'ownership_types',
       'qualities', 'page', 'new_transcription', 'is_people',  'llm_guess',
       ])
df[appreciated_order].to_csv('sommarioni_standardisation_pre_finished_20250512.csv', index=False)
df[appreciated_order].to_json('sommarioni_standardisation_pre_finished_20250512.json', orient='records', force_ascii=False)